In [5]:
import time
import os
import pandas as pd
import numpy as np
import random

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2
from echoai_pet_measurements.runCFRModel import runCFRModel, Videoconverter

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load list of example videos ###

In [6]:
cfr_data_root = os.path.normpath('/mnt/obi0/andreas/data/cfr')
predict_dir = os.path.join(cfr_data_root, 'predictions_echodata','FirstEchoEvents2repeat')
model = 'cfr_a4c_dgx-1_fc1_global_cfr_calc'
predict_file = os.path.join(predict_dir, 'cfr_models_30fps', model+'.parquet')
predict_df = pd.read_parquet(predict_file)
echo_df = predict_df[predict_df['model_name']==model]
display(echo_df.head(2))

,filename,model_output,dataset,predictions,model_name,checkpoint,dir,study,mrn,datetime,file_base,identifier,frame_time,number_of_frames,heart_rate,deltaX,deltaY,a2c,a2c_laocc,a2c_lvocc_s,a3c,a3c_laocc,a3c_lvocc_s,a4c,a4c_far,a4c_laocc,a4c_lvocc_s,a4c_rv,a4c_rv_laocc,a5c,apex,other,plax_far,plax_lac,plax_laz,plax_laz_ao,plax_plax,psax_avz,psax_az,psax_mv,psax_pap,rvinf,subcostal,suprasternal,year_month,study_full_time,institution,model,manufacturer,max_view,sum_views
0,48b4c0c643454479_4903a58051d59c1632a2417b35c5_...,global_cfr_calc,cfr,1.596508,cfr_a4c_dgx-1_fc1_global_cfr_calc,cfr_a4c_dgx-1_fc1_global_cfr_calc_chkpt_150.h5,/mnt/obi0/phi/echo/npyFiles/BWH/48b4/48b4c0c64...,48b4c0c643454479_4903a58051d59c1632a2417b35c5,31689912,2015-11-19 11:12:41,48b4c0c643454479_4903a58051d59c1632a2417b35c5_...,48b4c0c643454479_4903a58051d59c1632a2417b35c5_...,17.940552,158.0,68.0,0.044297,0.044297,1.211774e-10,5.782177e-03,3.122549e-08,5.130417e-10,1.875287e-09,5.872997e-09,0.989248,9.680495e-04,5.440455e-10,1.539974e-08,3.199503e-04,2.698714e-09,0.000007,4.936409e-07,8.984117e-08,9.937149e-09,8.138211e-07,6.816249e-10,2.948528e-07,2.625438e-10,3.655429e-03,1.264262e-06,1.696565e-08,0.000016,4.799932e-09,8.772870e-12,3.521227e-07,2015.0,20151119111241,BWH,Vivid E9,GE Vingmed Ultrasound,a4c,1.0
1,48b4c0c64adf5e8d_4903a58051d59c17833b0801157e_...,global_cfr_calc,cfr,2.103778,cfr_a4c_dgx-1_fc1_global_cfr_calc,cfr_a4c_dgx-1_fc1_global_cfr_calc_chkpt_150.h5,/mnt/obi0/phi/echo/npyFiles/BWH/48b4/48b4c0c64...,48b4c0c64adf5e8d_4903a58051d59c17833b0801157e,31680713,2015-11-18 09:27:43,48b4c0c64adf5e8d_4903a58051d59c17833b0801157e_...,48b4c0c64adf5e8d_4903a58051d59c17833b0801157e_...,33.333000,71.0,80.0,0.044037,0.044037,1.558442e-08,5.105510e-09,3.570098e-10,9.393454e-07,8.801211e-09,9.468467e-10,0.999441,2.660141e-09,3.068257e-13,5.856417e-11,5.877564e-08,5.032301e-10,0.000008,5.372147e-09,3.069948e-10,4.530195e-11,2.162873e-09,1.746480e-10,1.692652e-11,1.718629e-11,2.499897e-08,7.669211e-08,3.297388e-10,0.000549,3.299375e-07,1.455478e-10,7.087799e-09,2015.0,20151118092743,BWH,iE33,Philips Medical Systems,a4c,1.0


In [7]:
# pick 10 random videos
n_videos=10
np.random.seed(123)
file_list = np.random.choice(echo_df['filename'].unique(), size=n_videos, replace=False)
file_list = [os.path.join(echo_df[echo_df['filename']==file]['dir'].values[0], file) for file in file_list]

def parameter_list(echo_df, file_list, video_attribute):
    video_attribute_list = [echo_df[echo_df['filename']==os.path.basename(file)][video_attribute].values[0] \
                            for file in file_list]
    print(f'{len(video_attribute_list)} values for attribute {video_attribute} found.')
    return video_attribute_list

deltaX_list = parameter_list(echo_df=echo_df, file_list=file_list, video_attribute='deltaX')
deltaY_list = parameter_list(echo_df=echo_df, file_list=file_list, video_attribute='deltaY')
frame_time_list = parameter_list(echo_df=echo_df, file_list=file_list, video_attribute='frame_time')
VC = Videoconverter(max_frame_time_ms=None, min_frames=None, meta_df=None)
npy_array_list = [VC.load_video(file) for file in file_list]
print(f'{len(npy_array_list)} videos loaded.')

10 values for attribute deltaX found.
10 values for attribute deltaY found.
10 values for attribute frame_time found.
10 videos loaded.


### Models and weights ###

In [8]:
checkpoint_dir = os.path.normpath('/mnt/obi0/andreas/data/cfr/checkpoints')
model_output_list = ['global_cfr_calc', 'rest_global_mbf', 'stress_global_mbf']
checkpoint_list = ['global_cfr.h5', 'rest_mbf_ammonia.h5', 'stress_mbf_ammonia.h5']
checkpoint_file_list = [os.path.join(checkpoint_dir, checkpoint_file) for checkpoint_file in checkpoint_list]
checkpoint_dict = dict(zip(model_output_list, checkpoint_file_list))
display(checkpoint_dict)

{'global_cfr_calc': '/mnt/obi0/andreas/data/cfr/checkpoints/global_cfr.h5',
 'rest_global_mbf': '/mnt/obi0/andreas/data/cfr/checkpoints/rest_mbf_ammonia.h5',
 'stress_global_mbf': '/mnt/obi0/andreas/data/cfr/checkpoints/stress_mbf_ammonia.h5'}

### Predictions from earlier run ####

In [9]:
model_list = ['cfr_a4c_dgx-1_fc1_global_cfr_calc', 
              'mbf_ammonia_a4c_dgx-1_fc1_rest_global_mbf',
              'mbf_ammonia_a4c_dgx-1_fc1_stress_global_mbf']
pred_dict={}
for model in model_list:
    predict_file = os.path.join(predict_dir, 'cfr_models_30fps', model+'.parquet')
    predict_df = pd.read_parquet(predict_file, columns=['model_name', 'model_output', 'filename', 'predictions'])
    predict_df = predict_df[predict_df['model_name']==model]
    model_output = predict_df['model_output'].values[0]
    model_predictions = [predict_df[predict_df['filename']==os.path.basename(file)]['predictions'].values[0] for file in file_list]
    pred_dict_model = {model_output: model_predictions}
    pred_dict.update(pred_dict_model)
print(pred_dict.keys())

dict_keys(['global_cfr_calc', 'rest_global_mbf', 'stress_global_mbf'])


### Predictions from video list and checkpoints ###

In [10]:
qualified_index_list, predictions = runCFRModel(data_array_list=npy_array_list, 
                                                frame_time_ms_list=frame_time_list, 
                                                deltaX_list=deltaX_list, 
                                                deltaY_list=deltaY_list, 
                                                checkpoint_dict=checkpoint_dict,
                                                batch_size=1)

10/10 [==============================] - 1s 127ms/step


In [11]:
# Check the output
for model_output in checkpoint_dict.keys():
    df = pd.concat([pd.DataFrame(predictions[model_output]), pd.DataFrame(pred_dict[model_output])], axis=1)
    df.columns=['runCFRModel', 'earlier_predictions']
    display(df)

,runCFRModel,earlier_predictions
0,1.490338,1.490339
1,1.534409,1.534409
2,1.760748,1.760748
3,2.061305,2.061305
4,2.146248,2.146247
5,1.596508,1.596508
6,1.874179,1.874179
7,1.685252,1.685253
8,1.585242,1.585242
9,2.442383,2.442384


,runCFRModel,earlier_predictions
0,0.868456,0.868456
1,0.918252,0.918252
2,0.935816,0.935817
3,0.882128,0.882128
4,1.029121,1.029121
5,0.778684,0.778684
6,0.867392,0.867392
7,0.856307,0.856307
8,0.903970,0.903970
9,0.640754,0.640754


,runCFRModel,earlier_predictions
0,0.558606,0.558607
1,0.798279,0.798279
2,0.752713,0.752713
3,0.801471,0.801471
4,1.591175,1.591175
5,0.966645,0.966645
6,1.654421,1.654421
7,0.818925,0.818925
8,0.700806,0.700806
9,0.888685,0.888684
